In [ ]:
import sys

sys.path.append(".")

import json
import os
import re
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)
from torch.utils.data import DataLoader

torch.set_float32_matmul_precision('high')

# loading hyperparameters

model_dir = "models/2025-06-11 14:50/"

with open(model_dir + "hyperparameters.json", "r") as f:
    hyperparameters = json.loads(f.read())

# loading data

dataset = load_month_as_torch(
    2023,
    5,
    hyperparameters["h"],
    hyperparameters["f"],
    hyperparameters["quantile"]
)

test_loader = DataLoader(
    dataset,
    batch_size=hyperparameters["batch_size"],
    shuffle=True,
    pin_memory=True
)

X, _ = next(iter(test_loader))
in_channels = X.shape[1]

# loading optimal model

print("Loading optimal model...")

optimal_files = [f for f in os.listdir(model_dir) if re.search("latest", f)]
most_optimal = optimal_files[-1]

print(f"Found `{most_optimal}`")

model = TCNSimplified(
    in_channels=in_channels,
    lookback=hyperparameters["h"]
)
model.load_state_dict(torch.load(model_dir + most_optimal, weights_only=True))
model.eval()

# starting inference session

print("Starting inference...")

print()

all_preds_logits = []
all_preds_proba = []
all_true_labels = []

with torch.no_grad(): # disable gradient calculations for faster inference and less memory usage
    test_iterator = iter(test_loader)
    for i in range(min(1000, len(test_loader))):
        X_batch, y_batch = next(test_iterator)
    
        logits, loss = model(X_batch, y_batch)
        
        probabilities = torch.sigmoid(logits) 

        all_preds_logits.append(logits.cpu())
        all_preds_proba.append(probabilities.cpu())
        all_true_labels.append(y_batch.cpu())

all_preds_logits = torch.cat(all_preds_logits).numpy()
all_preds_proba = torch.cat(all_preds_proba).numpy()
all_true_labels = torch.cat(all_true_labels).numpy()
all_preds_hard = (all_preds_proba >= 0.5).astype(int)

plt.plot(figsize=(15, 8))
plt.grid(alpha=0.4)
plt.hist(all_preds_proba, bins=50)
plt.savefig("scrap.png")

loss_fn = nn.BCEWithLogitsLoss()

test_loss = loss_fn(torch.tensor(all_preds_logits), torch.tensor(all_true_labels)).item()
accuracy = accuracy_score(all_true_labels, all_preds_hard)
precision = precision_score(all_true_labels, all_preds_hard, average='binary')
recall = recall_score(all_true_labels, all_preds_hard, average='binary')
f1 = f1_score(all_true_labels, all_preds_hard, average='binary')
conf_matrix = confusion_matrix(all_true_labels, all_preds_hard)
cr = classification_report(all_true_labels, all_preds_hard, target_names=['Class 0', 'Class 1'])

print(f"Test Loss (BCEWithLogits): {test_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

try:
    roc_auc = roc_auc_score(all_true_labels, all_preds_proba)
    print(f"ROC AUC Score: {roc_auc:.4f}")
except ValueError as e:
    print(f"ROC AUC Score could not be calculated: {e}")
    print("This often happens if all true labels are the same (e.g., all 0s or all 1s).")

print("\nConfusion Matrix:")
print(conf_matrix)
print("  [[True Negatives, False Positives]")
print("   [False Negatives, True Positives]]")

print("\nClassification Report:")
print(cr)


ModuleNotFoundError: No module named 'sklearn'